# Customer Class

In [1]:
from collections import defaultdict
import datetime as dt

class Customer:
    def __init__(self, id, transactions):
        self.id = id
        
        self.transaction_dict = defaultdict(MonthYearTransaction)
        self.process_transactions(transactions)
    
    def process_transactions(self, transactions):
        for (index, date, amount) in transactions.itertuples():
            month_year = (date.month, date.year)
            self.transaction_dict[month_year].apply(amount)
            
    def generate_report(self):
        report = []
        for (month, year), transaction in self.transaction_dict.items():
            date_object = dt.datetime(year, month, 1)
            report.append([self.id, date_object.strftime('%m/%Y'), transaction.get_min_balance(), transaction.get_max_balance(), transaction.get_balance()])
        
        return report

# Transaction Class

In [2]:
import math

class MonthYearTransaction:
    def __init__(self):
        self.balance = 0
        self.min_balance = math.inf
        self.max_balance = -math.inf

    def apply(self, amount):
        self.balance += amount
        self.min_balance = min(self.min_balance, self.balance)
        self.max_balance = max(self.max_balance, self.balance)
    
    def get_balance(self):
        return self.balance
    
    def get_min_balance(self):
        return self.min_balance
    
    def get_max_balance(self):
        return self.max_balance

# Main function

In [3]:
import pandas as pd

In [16]:
df_transactions = pd.read_csv('input_random.csv', header=0, names=['customer_id', 'date', 'amount'])
customers = []


,customer_id,date,amount
0,C231,7/9/2020,-9232.0
1,C231,7/9/2020,6061.0
2,C231,5/6/2021,5108.0
3,C231,1/1/2022,9086.0
4,C231,1/9/2021,-8534.0
...,...,...,...
85,C409,11/26/2022,2285.0
86,C409,11/27/2022,-1209.0
87,C409,11/28/2022,-5432.0
88,NaN,NaN,NaN


In [17]:
# Data cleaning
df_transactions.dropna(inplace=True)

# Change the value of date column to type datetime for sorting
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

# Get all unique ids
customer_ids = df_transactions['customer_id'].unique()

In [ ]:
for id in customer_ids:
    # Get all transactions by a specific customer
    customer_transactions = df_transactions[df_transactions['customer_id'] == id][['date', 'amount']]
    
    # Sort his transactions by date and the amount because we want to apply credit transactions first
    customer_transactions.sort_values(['date', 'amount'], ascending=[True, False], inplace=True)
    
    # Create an instance of customer and append it to the customer list
    customers.append(Customer(id, customer_transactions))

In [ ]:
reports = []
for customer in customers:
    reports.extend(customer.generate_report())

pd.DataFrame(reports).to_csv('output.csv', header=False, index=False)